<a href="https://colab.research.google.com/github/abhi25072002/software-engineering-challenges/blob/master/WidsDatathon2024NeuralNetwork.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Artificial Neural Network + NLP,ensembling method

In [ ]:
!kaggle competitions download -c widsdatathon2024-challenge1

 79% 4.00M/5.04M [00:01<00:00, 4.44MB/s]
100% 5.04M/5.04M [00:01<00:00, 3.84MB/s]


In [ ]:
!unzip widsdatathon2024-challenge1.zip

Archive:  widsdatathon2024-challenge1.zip
  inflating: sample_submission.csv   
  inflating: test.csv                
  inflating: training.csv            


In [ ]:
import pandas as pd
train = pd.read_csv("./training.csv")
test = pd.read_csv("./test.csv")
print(train.isnull().sum())
print(test.isnull().sum())
print(train.columns)
train.drop(columns = ['patient_id'], inplace= True)
test.drop(columns = ['patient_id'], inplace= True)

patient_id           0
patient_race      6385
payer_type        1803
patient_state       51
patient_zip3         0
                  ... 
veteran              1
Ozone               29
PM25                29
N02                 29
DiagPeriodL90D       0
Length: 83, dtype: int64
patient_id             0
patient_race        2901
payer_type           760
patient_state         21
patient_zip3           0
                    ... 
health_uninsured       0
veteran                0
Ozone                 14
PM25                  14
N02                   14
Length: 82, dtype: int64
Index(['patient_id', 'patient_race', 'payer_type', 'patient_state',
       'patient_zip3', 'patient_age', 'patient_gender', 'bmi',
       'breast_cancer_diagnosis_code', 'breast_cancer_diagnosis_desc',
       'metastatic_cancer_diagnosis_code', 'metastatic_first_novel_treatment',
       'metastatic_first_novel_treatment_type', 'Region', 'Division',
       'population', 'density', 'age_median', 'age_under_10', 'age_10_t

In [ ]:
# describing Training data :
desc = pd.DataFrame(index = list(train))
desc['type'] = train.dtypes
desc['count'] = train.count()
desc['nunique'] = train.nunique()
desc['%unique'] = desc['nunique'] /len(train) * 100
desc['null'] = train.isnull().sum()
desc['%null'] = desc['null'] / len(train) * 100
desc = pd.concat([desc,train.describe().T.drop('count',axis=1)],axis=1)
desc.sort_values(by=['type','null']).style.background_gradient(axis=0)

In [ ]:
desc = pd.DataFrame(index = list(test))
desc['type'] = test.dtypes
desc['count'] = test.count()
desc['nunique'] = test.nunique()
desc['%unique'] = desc['nunique'] /len(test) * 100
desc['null'] = test.isnull().sum()
desc['%null'] = desc['null'] / len(test) * 100
desc = pd.concat([desc,test.describe().T.drop('count',axis=1)],axis=1)
desc.sort_values(by=['type','null']).style.background_gradient(axis=0)

In [ ]:
from sklearn.impute import SimpleImputer

# Define the imputers
median_imputer = SimpleImputer(strategy='median')
mode_imputer = SimpleImputer(strategy='most_frequent')
constant_imputer_novel_treatment = SimpleImputer(strategy='constant', fill_value='No')
constant_imputer_novel_treatment_type = SimpleImputer(strategy='constant', fill_value='Unknown')
constant_imputer_patient_race = SimpleImputer(strategy='constant', fill_value='Unknown')

# Impute 'bmi' with median
train[['bmi']] = median_imputer.fit_transform(train[['bmi']])

# Impute 'patient_race' as 'Unknown'
train[['patient_race']] = constant_imputer_patient_race.fit_transform(train[['patient_race']])

# Impute 'novel_treatment' as 'No' and 'novel_treatment_type' as 'Unknown'
train[['metastatic_first_novel_treatment']] = constant_imputer_novel_treatment.fit_transform(train[['metastatic_first_novel_treatment']])
train[['metastatic_first_novel_treatment_type']] = constant_imputer_novel_treatment_type.fit_transform(train[['metastatic_first_novel_treatment_type']])

# Impute 'payer_type' with the mode
train[['payer_type']] = mode_imputer.fit_transform(train[['payer_type']])

# Impute 'region', 'patient_state', and 'division' with the mode
columns_to_impute_mode = ['Region', 'patient_state', 'Division']
train[columns_to_impute_mode] = mode_imputer.fit_transform(train[columns_to_impute_mode])

# Impute 'ozone', 'NO2', 'PM2.5' with the median
columns_to_impute_median = ['Ozone', 'N02', 'PM25']
train[columns_to_impute_median] = median_imputer.fit_transform(train[columns_to_impute_median])

In [ ]:
from sklearn.impute import SimpleImputer

# Define the imputers
median_imputer = SimpleImputer(strategy='median')
mode_imputer = SimpleImputer(strategy='most_frequent')
constant_imputer_novel_treatment = SimpleImputer(strategy='constant', fill_value='No')
constant_imputer_novel_treatment_type = SimpleImputer(strategy='constant', fill_value='Unknown')
constant_imputer_patient_race = SimpleImputer(strategy='constant', fill_value='Unknown')

# Impute 'bmi' with median
test[['bmi']] = median_imputer.fit_transform(test[['bmi']])

# Impute 'patient_race' as 'Unknown'
test[['patient_race']] = constant_imputer_patient_race.fit_transform(test[['patient_race']])

# Impute 'novel_treatment' as 'No' and 'novel_treatment_type' as 'Unknown'
test[['metastatic_first_novel_treatment']] = constant_imputer_novel_treatment.fit_transform(test[['metastatic_first_novel_treatment']])
test[['metastatic_first_novel_treatment_type']] = constant_imputer_novel_treatment_type.fit_transform(test[['metastatic_first_novel_treatment_type']])

# Impute 'payer_type' with the mode
test[['payer_type']] = mode_imputer.fit_transform(test[['payer_type']])

# Impute 'region', 'patient_state', and 'division' with the mode
columns_to_impute_mode = ['Region', 'patient_state', 'Division']
test[columns_to_impute_mode] = mode_imputer.fit_transform(test[columns_to_impute_mode])

# Impute 'ozone', 'NO2', 'PM2.5' with the median
columns_to_impute_median = ['Ozone', 'N02', 'PM25']
test[columns_to_impute_median] = median_imputer.fit_transform(test[columns_to_impute_median])

In [ ]:
train['patient_zip3'] = train['patient_zip3'].astype('category')
test['patient_zip3'] = test['patient_zip3'].astype('category')


In [ ]:
for column in train.columns:
    # Check if the column is numerical (int or float)
    if train[column].dtype == 'float64' or train[column].dtype == 'int64':
        train[column].fillna(train[column].mean(), inplace=True)
    # Else, the column is categorical, fill with the mode
    else:
        train[column].fillna(train[column].mode()[0], inplace=True)

In [ ]:
for column in test.columns:
    # Check if the column is numerical (int or float)
    if test[column].dtype == 'float64' or test[column].dtype == 'int64':
        test[column].fillna(test[column].mean(), inplace=True)
    # Else, the column is categorical, fill with the mode
    else:
        test[column].fillna(test[column].mode()[0], inplace=True)


In [ ]:
#verify if all null values are hanlded in both traina dn test
remaining_nulls = train.isnull().sum()
print(remaining_nulls[remaining_nulls > 0])
remaining_nullst = test.isnull().sum()
print(remaining_nullst[remaining_nullst > 0])

Series([], dtype: int64)
Series([], dtype: int64)


In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

# Assuming 'train' is your DataFrame and 'numerical_columns' contains the names of all numerical columns
numerical_columns = train.select_dtypes(include=['float64', 'int64']).columns.tolist()

# Creating boxplots for each numerical column
for col in numerical_columns:
    plt.figure(figsize=(6, 6))  # Set the figure size for better visibility
    sns.boxplot(x=train[col])  # Create a boxplot for the column
    plt.title(f'Boxplot of {col}')  # Set the title of the boxplot
    plt.xlabel(col)  # Set the x-axis label as the column name
    plt.show()  # Display the plot


In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

# Assuming 'train' is your DataFrame and 'numerical_columns' contains the names of all numerical columns
numerical_columns = test.select_dtypes(include=['float64', 'int64']).columns.tolist()

# Creating boxplots for each numerical column
for col in numerical_columns:
    plt.figure(figsize=(6, 6))  # Set the figure size for better visibility
    sns.boxplot(x=test[col])  # Create a boxplot for the column
    plt.title(f'Boxplot of {col}')  # Set the title of the boxplot
    plt.xlabel(col)  # Set the x-axis label as the column name
    plt.show()  # Display the plot


In [ ]:
num_col = []
cat_col = []
for col in train.columns:
    if train[col].dtypes in ['float64' , 'int64']:
        num_col.append(col)
    else:
       cat_col.append(col)
print(num_col)
print(cat_col)
num_col = ['patient_age', 'bmi', 'population', 'density', 'age_median', 'age_under_10', 'age_10_to_19', 'age_20s', 'age_30s', 'age_40s', 'age_50s', 'age_60s', 'age_70s', 'age_over_80', 'male', 'female', 'married', 'divorced', 'never_married', 'widowed', 'family_size', 'family_dual_income', 'income_household_median', 'income_household_under_5', 'income_household_5_to_10', 'income_household_10_to_15', 'income_household_15_to_20', 'income_household_20_to_25', 'income_household_25_to_35', 'income_household_35_to_50', 'income_household_50_to_75', 'income_household_75_to_100', 'income_household_100_to_150', 'income_household_150_over', 'income_household_six_figure', 'income_individual_median', 'home_ownership', 'housing_units', 'home_value', 'rent_median', 'rent_burden', 'education_less_highschool', 'education_highschool', 'education_some_college', 'education_bachelors', 'education_graduate', 'education_college_or_above', 'education_stem_degree', 'labor_force_participation', 'unemployment_rate', 'self_employed', 'farmer', 'race_white', 'race_black', 'race_asian', 'race_native', 'race_pacific', 'race_other', 'race_multiple', 'hispanic', 'disabled', 'poverty', 'limited_english', 'commute_time', 'health_uninsured', 'veteran', 'Ozone', 'PM25', 'N02']

['patient_age', 'bmi', 'population', 'density', 'age_median', 'age_under_10', 'age_10_to_19', 'age_20s', 'age_30s', 'age_40s', 'age_50s', 'age_60s', 'age_70s', 'age_over_80', 'male', 'female', 'married', 'divorced', 'never_married', 'widowed', 'family_size', 'family_dual_income', 'income_household_median', 'income_household_under_5', 'income_household_5_to_10', 'income_household_10_to_15', 'income_household_15_to_20', 'income_household_20_to_25', 'income_household_25_to_35', 'income_household_35_to_50', 'income_household_50_to_75', 'income_household_75_to_100', 'income_household_100_to_150', 'income_household_150_over', 'income_household_six_figure', 'income_individual_median', 'home_ownership', 'housing_units', 'home_value', 'rent_median', 'rent_burden', 'education_less_highschool', 'education_highschool', 'education_some_college', 'education_bachelors', 'education_graduate', 'education_college_or_above', 'education_stem_degree', 'labor_force_participation', 'unemployment_rate', 'self

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import make_column_transformer
from sklearn.impute import SimpleImputer
from sklearn.pipeline import make_pipeline
### Here combine both dfs
preprocessor = make_column_transformer(
    (make_pipeline(StandardScaler()), num_col),
    (OneHotEncoder(), cat_col)
)

In [ ]:
y = train['DiagPeriodL90D']

In [ ]:
train_X = train.drop(columns=['DiagPeriodL90D'])
test_X = test
print(train_X.shape, test_X.shape)


(12906, 81) (5792, 81)


In [ ]:
whole_df = pd.concat([train_X, test_X], axis =0)
whole_sparse = preprocessor.fit_transform(whole_df)
whole_dense = whole_sparse.toarray()
print(whole_dense.shape)

(18698, 1056)


In [ ]:
train_X_dense = whole_dense[:12906, :]
test_X_dense = whole_dense[12906:, :]

In [ ]:
print(train_X_dense.shape, test_X_dense.shape)

(12906, 1056) (5792, 1056)


In [ ]:
!pip install optuna

In [ ]:
import optuna
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.optimizers import Adam, SGD, RMSprop
from sklearn.model_selection import train_test_split
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

# Your data split
X_train, X_test, y_train, y_test = train_test_split(train_X_dense, y, test_size=0.25, stratify=y, random_state=66)

def create_model(trial):
    # Hyperparameters to be tuned
    n_layers = trial.suggest_int('n_layers', 1, 3)
    activation = trial.suggest_categorical('activation', ['relu', 'tanh', 'sigmoid'])
    dropout_rate = trial.suggest_float('dropout_rate', 0.1, 0.5)
    optimizer = trial.suggest_categorical('optimizer', ['adam', 'sgd', 'rmsprop'])
    learning_rate = trial.suggest_loguniform('learning_rate', 1e-5, 1e-1)

    # Model architecture
    model = Sequential()
    model.add(Dense(trial.suggest_int('units_first', 32, 256), activation=activation, input_dim=X_train.shape[1]))
    model.add(Dropout(dropout_rate))
    for i in range(n_layers):
        num_units = trial.suggest_int(f'units_layer_{i+1}', 32, 256)
        model.add(Dense(num_units, activation=activation))
        model.add(Dropout(dropout_rate))
    model.add(Dense(1, activation='sigmoid'))  # Output layer for binary classification

    # Compilation
    if optimizer == 'adam':
        opt = Adam(learning_rate=learning_rate)
    elif optimizer == 'sgd':
        opt = SGD(learning_rate=learning_rate)
    elif optimizer == 'rmsprop':
        opt = RMSprop(learning_rate=learning_rate)

    model.compile(optimizer=opt, loss='binary_crossentropy', metrics=['accuracy'])

    return model

def objective(trial):
    model = create_model(trial)

    # Callbacks
    early_stopping = EarlyStopping(monitor='val_accuracy', mode='max', patience=5)

    # Fit the model
    history = model.fit(
        X_train,
        y_train,
        validation_split=0.25,
        epochs=50,  # May reduce for faster experimentation
        batch_size=32,
        callbacks=[early_stopping],
        verbose=0  # Set to 1 if you want to see the output
    )

    # Evaluate the model
    loss, accuracy = model.evaluate(X_test, y_test, verbose=0)
    return accuracy

study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=50)  # Increase n_trials for a more thorough search

# Best trial results
print(f'Number of finished trials: {len(study.trials)}')
print(f'Best trial:')
trial = study.best_trial

print(f'  Value: {trial.value}')
print(f'  Params: ')
for key, value in trial.params.items():
    print(f'    {key}: {value}')

# Rebuild and retrain the best model
best_model = create_model(study.best_trial)
best_model.fit(X_train, y_train, epochs=100, batch_size=32, verbose=1)  # Retrain with more epochs if needed

# Evaluate the best model
loss, accuracy = best_model.evaluate(X_test, y_test)
print(f'Best model accuracy: {accuracy}')


[I 2024-02-24 20:39:20,915] A new study created in memory with name: no-name-4f7e0c2e-e859-4ae8-9934-3ae048208d83
<ipython-input-38-1ca4adbe2711>:18: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  learning_rate = trial.suggest_loguniform('learning_rate', 1e-5, 1e-1)
[I 2024-02-24 20:39:32,712] Trial 0 finished with value: 0.6244189739227295 and parameters: {'n_layers': 2, 'activation': 'sigmoid', 'dropout_rate': 0.21011389411491288, 'optimizer': 'sgd', 'learning_rate': 0.07749370961071209, 'units_first': 132, 'units_layer_1': 104, 'units_layer_2': 103}. Best is trial 0 with value: 0.6244189739227295.
[I 2024-02-24 20:40:23,400] Trial 1 finished with value: 0.8044623732566833 and parameters: {'n_layers': 1, 'activation': 'tanh', 'dropout_rate': 0.31904654598718585, 'optimizer': 'sgd', 'learning_rate': 0.0031079964391018283, 'unit

Number of finished trials: 50
Best trial:
  Value: 0.8097304105758667
  Params: 
    n_layers: 1
    activation: tanh
    dropout_rate: 0.13252729840177874
    optimizer: rmsprop
    learning_rate: 2.7793301752452672e-05
    units_first: 117
    units_layer_1: 178
Epoch 1/100
303/303 [==============================] - 2s 3ms/step - loss: 0.6632 - accuracy: 0.6043
Epoch 2/100
303/303 [==============================] - 1s 3ms/step - loss: 0.6165 - accuracy: 0.6718
Epoch 3/100
303/303 [==============================] - 1s 3ms/step - loss: 0.5853 - accuracy: 0.7301
Epoch 4/100
303/303 [==============================] - 1s 4ms/step - loss: 0.5561 - accuracy: 0.7643
Epoch 5/100
303/303 [==============================] - 2s 5ms/step - loss: 0.5276 - accuracy: 0.7845
Epoch 6/100
303/303 [==============================] - 1s 4ms/step - loss: 0.5099 - accuracy: 0.7935
Epoch 7/100
303/303 [==============================] - 1s 3ms/step - loss: 0.4981 - accuracy: 0.7967
Epoch 8/100
303/303 [=======

In [ ]:
# Assuming train_X_dense and y are your entire training dataset and labels
best_model.fit(train_X_dense, y, epochs=100, batch_size=64, verbose=1)  # You might adjust epochs and batch_size

test_pred_probs = best_model.predict(test_X_dense)
t1= pd.read_csv('./test.csv')
print(test_pred_probs.shape, t1['patient_id'].shape)

test_pred_probs_1d = test_pred_probs.flatten()
# Assuming you have an ID column in your original test dataset to match the predictions
# Create a DataFrame with IDs and the predicted probabilities
submission_df1 = pd.DataFrame({
    'patient_id': t1['patient_id'],
    'DiagPeriodL90D': test_pred_probs_1d
})

# Save the DataFrame to a CSV file
submission_df1.to_csv('submission.csv', index=False)


Epoch 1/100
202/202 [==============================] - 2s 8ms/step - loss: 0.4578 - accuracy: 0.8122
Epoch 2/100
202/202 [==============================] - 1s 7ms/step - loss: 0.4581 - accuracy: 0.8116
Epoch 3/100
202/202 [==============================] - 1s 6ms/step - loss: 0.4583 - accuracy: 0.8116
Epoch 4/100
202/202 [==============================] - 1s 6ms/step - loss: 0.4567 - accuracy: 0.8118
Epoch 5/100
202/202 [==============================] - 1s 6ms/step - loss: 0.4562 - accuracy: 0.8130
Epoch 6/100
202/202 [==============================] - 1s 4ms/step - loss: 0.4568 - accuracy: 0.8104
Epoch 7/100
202/202 [==============================] - 1s 4ms/step - loss: 0.4566 - accuracy: 0.8124
Epoch 8/100
202/202 [==============================] - 1s 4ms/step - loss: 0.4565 - accuracy: 0.8131
Epoch 9/100
202/202 [==============================] - 1s 3ms/step - loss: 0.4568 - accuracy: 0.8111
Epoch 10/100
202/202 [==============================] - 1s 3ms/step - loss: 0.4565 - accura

In [ ]:
# X = train_df.drop('DiagPeriodL90D', axis=1)  # Replace 'diagonal90D' with your target column name
# y = train_df['DiagPeriodL90D']

In [ ]:
# Predict probabilities on the test dataset
test_pred_probs = best_model.predict(test_X_dense)
t1= pd.read_csv('./test.csv')
print(test_pred_probs.shape, t1['patient_id'].shape)

test_pred_probs_1d = test_pred_probs.flatten()
# Assuming you have an ID column in your original test dataset to match the predictions
# Create a DataFrame with IDs and the predicted probabilities
submission_df1 = pd.DataFrame({
    'patient_id': t1['patient_id'],
    'DiagPeriodL90D': test_pred_probs_1d
})

# Save the DataFrame to a CSV file
submission_df1.to_csv('submission.csv', index=False)


181/181 [==============================] - 0s 2ms/step
(5792, 1) (5792,)


In [ ]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.optimizers import Adam
from sklearn.metrics import roc_auc_score
import numpy as np

# Assuming 'X' and 'y' are your features and labels respectively
X_train, X_test, y_train, y_test = train_test_split(train_X_dense, y, test_size=0.25, stratify=y, random_state=66)

# Define the model
model = Sequential([
    Dense(128, input_dim=X_train.shape[1], activation='relu'),
    Dropout(0.2),
    Dense(64, activation='relu'),
    Dropout(0.2),
    Dense(32, activation='relu'),
    Dense(1, activation='sigmoid')  # Output layer for binary classification
])

learning_rate = 0.001  # You can change this to whatever value you wish

# Create an instance of the Adam optimizer with the desired learning rate
adam_optimizer = Adam(learning_rate=learning_rate)

# Compile the model with additional metrics
model.compile(optimizer=adam_optimizer,
              loss='binary_crossentropy',
              metrics=[tf.keras.metrics.AUC(name='auc'), 'accuracy'])

# Callbacks for early stopping (to prevent overfitting) and best model checkpointing
# early_stopping = EarlyStopping(monitor='val_auc', mode='max', verbose=1, patience=10)
model_checkpoint = ModelCheckpoint('best_model.h5', monitor='val_auc', mode='max', save_best_only=True, verbose=1)

# Train the model
history = model.fit(X_train, y_train,
                    validation_split=0.25,
                    epochs=100,  # Adjust based on the convergence of your model
                    batch_size=32,  # Adjust based on your preference and GPU memory
                    callbacks=[model_checkpoint],
                    verbose=1)

# Load the best model saved by ModelCheckpoint
best_model = tf.keras.models.load_model('best_model.h5')

# Evaluate the model on the test data
loss, auc, accuracy = best_model.evaluate(X_test, y_test)
print(f'Test Loss: {loss}, Test AUC: {auc}, Test Accuracy: {accuracy}')

# Predict probabilities on the test set for ROC-AUC score
y_pred_probs = best_model.predict(X_test)
roc_auc = roc_auc_score(y_test, y_pred_probs)
print(f'Test ROC-AUC: {roc_auc}')


In [ ]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.callbacks import ModelCheckpoint
from sklearn.model_selection import train_test_split, StratifiedKFold
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.optimizers import Adam
from sklearn.metrics import roc_auc_score
import numpy as np

# Assuming 'X' and 'y' are your features and labels respectively
# Make sure that 'train_X_dense' and 'y' are numpy arrays before you proceed
# If they are not, convert them using np.array()

n_splits = 5  # Number of folds
kfold = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=66)
test_predictions = np.zeros((test_X_dense.shape[0],))

early_stopping = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=10)
model_checkpoint = ModelCheckpoint('best_model.h5', monitor='val_loss', mode='min', save_best_only=True, verbose=1)

for train, test in kfold.split(train_X_dense, y):
    # Define the model architecture
    model = Sequential([
        Dense(128, input_dim=train_X_dense.shape[1], activation='relu'),
        Dropout(0.2),
        Dense(64, activation='relu'),
        Dropout(0.2),
        Dense(32, activation='relu'),
        Dense(1, activation='sigmoid')  # Output layer for binary classification
    ])

    learning_rate = 0.001  # Adjust learning rate as needed
    adam_optimizer = Adam(learning_rate=learning_rate)
    model.compile(optimizer=adam_optimizer,
                  loss='binary_crossentropy',
                  metrics=[tf.keras.metrics.AUC(name='auc'), 'accuracy'])

    # Fit the model
    history = model.fit(train_X_dense[train], y[train],
                        validation_data=(train_X_dense[test], y[test]),
                        callbacks=[early_stopping, model_checkpoint],
                        epochs=100,  # Adjust based on the convergence of your model
                        batch_size=64,  # Adjust based on your preference and GPU memory
                        verbose=1)

    # Evaluate the model
    scores = model.evaluate(train_X_dense[test], y[test], verbose=0)
    test_pred = model.predict(test_X_dense)
    test_predictions += test_pred.flatten()


test_predictions /= n_splits

# Optionally, compute a weighted average instead of a simple mean
# weights = val_scores / np.sum(val_scores)
# weighted_test_predictions = np.dot(test_predictions, weights)

# Prepare the submission file
t1 = pd.read_csv('./test.csv')  # Ensure this points to your test data
submission_df = pd.DataFrame({
    'patient_id': t1['patient_id'],
    'DiagPeriodL90D': test_predictions  # Or use weighted_test_predictions
})
submission_df.to_csv('weighted_submission.csv', index=False)


In [ ]:
import pandas as pd

# Load the submission files
sub1 = pd.read_csv('submission.csv')  # Assume this has columns 'patient_id' and 'DiagPeriodL90D'
sub2 = pd.read_csv('submission (5).csv')  # Same structure
# sub3 = pd.read_csv('weighted_submission.csv')  # Same structure

# Check that 'patient_id' columns match across all submissions
# This is crucial to ensure that predictions correspond to the same patients
assert all(sub1['patient_id'] == sub2['patient_id']) #and all(sub1['patient_id'] == sub3['patient_id'])

# Compute the weighted average of predictions
final_predictions = 0.6 * sub1['DiagPeriodL90D'] + 0.4 * sub2['DiagPeriodL90D'] #+ 0.2 * sub3['DiagPeriodL90D']

# Create the final submission file
final_submission = pd.DataFrame({
    'patient_id': sub1['patient_id'],
    'DiagPeriodL90D': final_predictions
})

# Save the final submission file
final_submission.to_csv('final_submission.csv', index=False)


(12906, 292)